In [1]:
# reload modules in .py files
%load_ext autoreload
%autoreload 2

In [2]:
# pull repo
!git clone https://github.com/char-tan/sparsity

Cloning into 'sparsity'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 49 (delta 17), reused 36 (delta 10), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [4]:
# change working directory
import os
os.chdir('sparsity/ops')

In [8]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing ops_cpp.egg-info/PKG-INFO
writing dependency_links to ops_cpp.egg-info/dependency_links.txt
writing top-level names to ops_cpp.egg-info/top_level.txt
/usr/local/lib/python3.8/dist-packages/torch/utils/cpp_extension.py:476: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
writing manifest file 'ops_cpp.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'ops_cpp' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.8/dist-packages/torch/include -I/usr/local/lib

In [9]:
# Using -e has caveats: https://stackoverflow.com/a/20043907
!pip3 install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/sparsity/ops
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ops-cpp: filename=ops_cpp-0.0.0-cp38-cp38-linux_x86_64.whl size=2327601 sha256=2c7ca32a672e7ce987b735593ba347b114a44845fb6da26d46442596f3a9d839
  Stored in directory: /tmp/pip-ephem-wheel-cache-2ta0lf1b/wheels/a0/60/38/cf329a695010b267296cd00c31c2a180e9d57ca424280b3d28
Successfully built ops-cpp
  Attempting uninstall: ops-cpp
    Found existing installation: ops-cpp 0.0.0
    Uninstalling ops-cpp-0.0.0:
      Successfully uninstalled ops-cpp-0.0.0

### Sample usage of our extension

In [12]:
import torch
import os, sys
sys.path.append(os.getcwd())
import ops_cpp

a = torch.tensor([1])
b = a
c = ops_cpp.add(a, b)
print(c)

ImportError: ignored

In [13]:
!git diff

diff --git a/ops/setup.py b/ops/setup.py
index 8e3156e..d83651d 100644
--- a/ops/setup.py
+++ b/ops/setup.py
@@ -2,5 +2,5 @@ from setuptools import setup, Extension
 from torch.utils import cpp_extension
 
 setup(name='ops_cpp',
-      ext_modules=[cpp_extension.CppExtension('ops_cpp', ['add.cpp', 'ops.cpp'])],
+      ext_modules=[cpp_extension.CppExtension('ops_cpp', ['ops_cuda.cpp'])],
       cmdclass={'build_ext': cpp_extension.BuildExtension})
